In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass 
from pathlib import Path
from typing import Union


@dataclass(frozen=True)
class CvToJsonConfig:
    cv_path: Union[Path, str]
    example_json_path: Union[Path, str]
    edited_json_path : Union[Path,str]
    model: str
    cv_to_json_prompt_templates:dict

    def __post_init__(self):
        # Convert string paths to Path objects
        object.__setattr__(self, 'cv_path', Path(self.cv_path))
        object.__setattr__(self, 'example_json_path', Path(self.example_json_path))
        object.__setattr__(self, 'edited_json_path', Path(self.edited_json_path))
       


In [4]:
import json

from PyPDF2 import PdfReader
from pathlib import Path
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

from job_aplication_agent.constants import *
from job_aplication_agent import logger
from job_aplication_agent.utils.common import  load_json,read_yaml,read_file,save_file

In [5]:
class ConfigurationManager: 
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            models_filepath = MODELS_FILE_PATH,
            prompt_template = PROMPT_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.models = read_yaml(models_filepath)
            self.prompt_template = read_yaml(prompt_template)

    def get_cv_to_json_config(self) -> CvToJsonConfig:
          create_cv_to_json_config = CvToJsonConfig(
                cv_path = self.config.Cv_to_json.cv_path,
                example_json_path = self.config.Cv_to_json.example_json_path,
                edited_json_path = self.config.Cv_to_json.edited_json_path,
                model= self.models.Llama3,
                cv_to_json_prompt_templates = self.prompt_template.cv_to_json_prompt_templates
                )
          return create_cv_to_json_config
          
    


In [6]:
class GetCvToJson:
    def __init__(self, config:CvToJsonConfig):
        self.config = config
        
    def create_cv_to_json_agent(self):
        llm = ChatGroq(temperature=0,model_name = self.config.model)
        logger.info(f"create_cv_to_json_agent Llm initialized")
        try: 
            cv_pdf_to_json_prompt = PromptTemplate(
                template=self.config.cv_to_json_prompt_templates.cv_to_json_prompt_template,
                input_variables=["CV", "JSON"]
            )
            self.cv_to_json_agent = cv_pdf_to_json_prompt | llm | JsonOutputParser()
            logger.info("Cv to Json Agent created")
        except Exception as e:
            logger.error(f"Creating Cv to Json Agent error: {e}")
    
    def get_cv(self):
        try: 
            with open(self.config.cv_path, 'rb') as file:
                reader = PdfReader(file)
            
                # Loop through all the pages and extract text
                cv_content = []
                for i, page in enumerate(reader.pages):
                    content = page.extract_text()
                    cv_content.append(content)
            
            full_cv_content = ''.join(cv_content)
            logger.info(f"Cv loaded")
            return full_cv_content
        except Exception as e:
            logger.error(f"Error loading cv from {self.config.cv_path} with error {e}")
        
        

    def get_json(self):
        try:
            example_json = load_json(self.config.example_json_path)
            example_json_str = json.dumps(example_json)
            logger.info(f"Json example loaded")
            return example_json_str
        except Exception as e:
            logger.error(f"Error loading json from {self.config.example_json_path} with error {e}")
        

    def get_cv_to_json(self,CV,JSON):
        try:
            cv_to_json = self.cv_to_json_agent.invoke({
                "CV":CV,
                "JSON":JSON
                })
            logger.info(f"Cv to json succefully obtained")
            # Saving the dictionary to a JSON file
            with open(self.config.edited_json_path, 'w') as json_file:
                json.dump(cv_to_json, json_file, indent=4)
            logger.info(f"Edited json saved in {self.config.edited_json_path}")
        except Exception as e:
            logger.error(f"Error getting cv to json {e}")
        
        
        
        

        


In [7]:
config = ConfigurationManager()
cv_to_json_config = config.get_cv_to_json_config()
cv_to_json = GetCvToJson(cv_to_json_config)
CV = cv_to_json.get_cv()
JSON = cv_to_json.get_json()
cv_to_json.create_cv_to_json_agent()
get_cv_to_json = cv_to_json.get_cv_to_json(CV,JSON)


[2024-09-04 12:02:56,016: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-09-04 12:02:56,019: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-09-04 12:02:56,029: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
[2024-09-04 12:02:56,117: INFO: 2345879886: Cv loaded:]
[2024-09-04 12:02:56,120: INFO: common: json file loaded succesfully from: Cv_to_json\cvjson.json:]
[2024-09-04 12:02:56,122: INFO: 2345879886: Json example loaded:]
[2024-09-04 12:02:56,983: INFO: 2345879886: create_cv_to_json_agent Llm initialized:]
[2024-09-04 12:02:56,985: INFO: 2345879886: Cv to Json Agent created:]
[2024-09-04 12:03:00,648: INFO: _client: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK":]
[2024-09-04 12:03:00,663: INFO: 2345879886: Cv to json succefully obtained:]
[2024-09-04 12:03:00,666: INFO: 2345879886: Edited json saved in Cv_to_json\edited_json.json:]
